In [1]:
from fastai.vision import *
import onnx
import onnxruntime
from PIL import Image

In [2]:
learn = load_learner('.')
fastai_model = learn.model.eval()

In [3]:
imagenet_mean, imagenet_std = torch.tensor([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])

In [4]:
class ImageScale(nn.Module):
    def forward(self, x):
        xx = (x/255.0-imagenet_mean[...,None,None]) / imagenet_std[...,None,None] 
        return xx.unsqueeze(0)
    
scale_layer = ImageScale()
softmax_layer = torch.nn.Softmax(dim=1)

In [5]:
model_name = 'pneumonia.onnx'
session = onnxruntime.InferenceSession(model_name)

In [6]:
image = Image.open('pneumonia.jpeg')
image = image.resize((256, 256))
image = image.convert(mode='RGB')
image_data = np.array(image).transpose(2, 0, 1).astype('float32')

In [7]:
input_name = session.get_inputs()[0].name
onnx_result = session.run([], {input_name: image_data})
onnx_result

[array([[0.056997, 0.943003]], dtype=float32)]

In [8]:
image_tensor = torch.tensor(image_data)
fastai_model = learn.model.eval()

with torch.no_grad():
    input_tensor = scale_layer(image_tensor)
    fastai_result = fastai_model(input_tensor)
    fastai_result = softmax_layer(fastai_result)

fastai_result

tensor([[0.0570, 0.9430]])

In [15]:
fastai_img = open_image('pneumonia.jpeg')
learn.predict(fastai_img)

(Category PNEUMONIA, tensor(1), tensor([0.0013, 0.9987]))

In [16]:
def preprocess(img, sz):
    _, r, c = img.shape
    ratio = min(r/sz, c/sz)
    r, c = int(round(r/ratio)), int(round(c/ratio))
    img = img.resize((3, r, c)).data
    
    row = int((r-sz+1)*0.5)
    col = int((c-sz+1)*0.5)
    img = img[:, row:row+sz, col:col+sz]    
    return img

In [17]:
fastai_img = preprocess(fastai_img, 256) * 255.0
with torch.no_grad():
    input_tensor = scale_layer(fastai_img)
    result = fastai_model(input_tensor)
    result = softmax_layer(result)[0]
    
result

tensor([0.0013, 0.9987])

In [23]:
onnx_result = session.run([], {input_name: fastai_img.numpy()})
onnx_result

[array([[0.001281, 0.998719]], dtype=float32)]

In [24]:
image = Image.open('pneumonia.jpeg')
image = image.convert(mode='RGB')
image.size

(1106, 762)

In [25]:
sz = 256
r, c = image.size
ratio = min(r/sz, c/sz)
r, c = int(round(r/ratio)), int(round(c/ratio))
image = image.resize((r, c), resample = Image.BILINEAR)
image.size

(372, 256)

In [26]:
image_data = np.array(image).transpose(2, 0, 1).astype('float32')
image_data.shape

(3, 256, 372)

In [27]:
r, c = c, r
row = int((r-sz+1)*0.5)
col = int((c-sz+1)*0.5)
image_data = image_data[:, row:row+sz, col:col+sz]
image_data.shape

(3, 256, 256)

In [28]:
onnx_result = session.run([], {input_name: image_data})
onnx_result

[array([[0.007556, 0.992444]], dtype=float32)]